In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tc import ThermoCycler
tc = ThermoCycler(ip="169.254.151.99")
res = tc.open_door()

<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/"> <s:Body> <OpenDoor xmlns="http://sila.coop" xmlns:i="http://www.w3.org/2001/XMLSchema-instance"> <requestId>980077707</requestId> <lockId i:nil="true"/> </OpenDoor> </s:Body> </s:Envelope>
<?xml version="1.0" encoding="UTF-8"?>
<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:SOAP-ENC="http://schemas.xmlsoap.org/soap/encoding/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:i="http://inheco.com" xmlns:s="http://sila.coop"><SOAP-ENV:Body><s:OpenDoorResponse><s:OpenDoorResult><s:returnCode>2</s:returnCode><s:message>Asynchronous command accepted.</s:message><s:duration>PT14S</s:duration><s:deviceClass>30</s:deviceClass></s:OpenDoorResult></s:OpenDoorResponse></SOAP-ENV:Body></SOAP-ENV:Envelope>



In [10]:
await tc.open_door()

<Response [200]>

In [ ]:
res = tc.close_door()
res.status_code

In [ ]:
# http && !(http contains "GetStatus") && !(http contains "Continue")


In [ ]:
import xml.etree.ElementTree as ET
import time
namespaces = {
    'SOAP-ENV': 'http://schemas.xmlsoap.org/soap/envelope/',
    's': 'http://sila.coop'
}

for i in range(30):
  print("wait ",i)
  time.sleep(1)
  a =tc.get_status1()
  root = ET.fromstring(a.text)
  print(root.find('.//s:state', namespaces).text)

In [3]:
tc.get_state()

<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/"> <s:Body> <GetStatus xmlns="http://sila.coop" xmlns:i="http://www.w3.org/2001/XMLSchema-instance"> <requestId>980077708</requestId> <lockId i:nil="true"/> </GetStatus> </s:Body> </s:Envelope>
<?xml version="1.0" encoding="UTF-8"?>
<SOAP-ENV:Envelope xmlns:SOAP-ENV="http://schemas.xmlsoap.org/soap/envelope/" xmlns:SOAP-ENC="http://schemas.xmlsoap.org/soap/encoding/" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:i="http://inheco.com" xmlns:s="http://sila.coop"><SOAP-ENV:Body><s:GetStatusResponse><s:GetStatusResult><s:returnCode>1</s:returnCode><s:message>Success.</s:message><s:duration>PT1S</s:duration><s:deviceClass>30</s:deviceClass></s:GetStatusResult><s:deviceId>97122c37-ab8b-4072-89aa-53f9715e0d3a</s:deviceId><s:state>idle</s:state><s:subStates xsi:nil="true"/><s:locked>false</s:locked><s:PMSId>http://169.254.193.225:7071/ihc</s:PMSId><s:currentTime>2024-09-26T1

<DeviceStatus.IDLE: 'idle'>

In [ ]:
tc.close_door()

In [ ]:
import requests
# POST / HTTP/1.1
# Content-Type: text/xml; charset=utf-8
# SOAPAction: "http://sila.coop/CloseDoor"
# Host: 169.254.245.237:8080
# Content-Length: 249
# Expect: 100-continue

requests.post("http://10.8.8.1:8080/",
              data='<s:Envelope xmlns:s="http://schemas.xmlsoap.org/soap/envelope/"><s:Body><OpenDoor xmlns="http://sila.coop" xmlns:i="http://www.w3.org/2001/XMLSchema-instance"><requestId>1579489236</requestId><lockId i:nil="true"/></OpenDoor></s:Body></s:Envelope>',
              headers={"Content-Type": "text/xml; charset=utf-8",
                        "SOAPAction": "http://sila.coop/OpenDoor",
                        "Host": "10.8.8.1:8080",
              }, timeout=1).text